In [1]:
from scipy import linalg, mat, dot
import numpy as np
import scipy as sp
from numpy.random import random
import os
import numpy as np
import pandas as pd
import tqdm
import time
import math

dir_ = './data/'
file_name = 'normalized_filter_track_5_user_100.csv'

In [2]:
# B = [[3,7,5,1],[5,3,8,3],[2,4,6,9]]
# A = mat(test_matrix) 
# U, s, V = linalg.svd(A) 
# S = linalg.diagsvd(s,len(U),len(V)) 
# RA = dot(dot(U,S),V) 
#------------------------------------------
# from scipy.sparse import csc_matrix
# from scipy.sparse.linalg import svds
# B = [[1,1,4,5],
#      [1,0,5,4],
#      [5,4,0,1],
#      [4,5,1,1]]
# C = mat(B, dtype=float)
# u, s, vt = svds(C, k=40) # k is the number of factors

In [3]:
train = np.loadtxt(os.path.join(dir_, 'rating_matrix_' + file_name), delimiter=',')
test = np.loadtxt(os.path.join(dir_, 'test_list_' + file_name), delimiter=',')

In [4]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
M = csc_matrix(train, dtype=float)
k = 100
u, s, vt = svds(M, k=k) # k is the number of factors
S = linalg.diagsvd(s,100,100) 

In [5]:
print(len(u), len(u[0]))
print(len(vt), len(vt[0]))
print(len(S))
# np.set_printoptions(suppress=True)
# print(u, '\n\n', S, '\n\n', vt)
# RA = dot(dot(u,S),vt) 
# RA
# np.set_printoptions(suppress=True)
# print(train)
# print(len(train), len(train[0]))

220 100
100 54969
100


In [6]:
# print(vt)
# print(len(vt), len(vt[0]))
vt = vt.transpose()
# print(vt)
# print(len(vt), len(vt[0]))

In [7]:
def pearson_correlation(tid1, tid2):
        result = 0.0
        track1_data = vt[tid1]
        track2_data = vt[tid2]

        rx_avg = item_average_rating(track1_data)
        ry_avg = item_average_rating(track2_data)
        
        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for i in range(len(track1_data)):
            rxs = track1_data[i]
            rys = track2_data[i]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return result

def item_average_rating(track):
    avg_rating = 0.0
    for i in track:
        avg_rating += i
    avg_rating /= len(track) * 1.0
    return avg_rating

In [8]:
def knn(tid, k):
    neighbors = []
    result = []
    for track in range(len(vt)):
        if tid == track:
            continue
        upc = pearson_correlation(tid, track)
        neighbors.append([track, upc])
    sorted_neighbors = sorted(neighbors, key=lambda neighbors: (neighbors[1], neighbors[0]), reverse=True)   # - for desc sort

    for i in range(k):
        if i >= len(sorted_neighbors):
            break
        result.append(sorted_neighbors[i])
    return result

In [9]:
def predict_by_knn(uid, tid, k_nearest_neighbors):
        valid_neighbors = check_neighbors_validattion(uid, k_nearest_neighbors)
        ##################################
        if not len(valid_neighbors):
            return -1
        ##################################
        top_result = 0.0
        bottom_result = 0.0
        for neighbor in valid_neighbors:
            neighbor_id = neighbor[0]
            neighbor_similarity = neighbor[1]   # Wi1
            rating = train[uid][neighbor_id] # rating i,item
            top_result += neighbor_similarity * rating
            bottom_result += neighbor_similarity
        result = top_result/bottom_result
        return result

def check_neighbors_validattion(uid, k_nearest_neighbors):
    result = []
    for neighbor in k_nearest_neighbors:
        neighbor_id = neighbor[0]
        if train[uid][neighbor_id] > 0:
            result.append(neighbor)
    return result


In [10]:
def predict(uid, tid, k):
    
    if len(knn_list[tid]) == 0:
        k_nearest_neighbors = knn(tid, k)
        knn_list[tid] = k_nearest_neighbors
    else:
        k_nearest_neighbors = knn_list[tid]

    prediction = predict_by_knn(uid, tid, k_nearest_neighbors)
    return prediction

In [11]:
knn_list = []

for i in range(len(vt)):
    knn_list.append([])

In [ ]:
k = 100
prediction_list = []

for i in tqdm.tqdm_notebook(range(len(test))):
    
    target_user = int(test[i][0])
    target_item = int(test[i][1])
    
    prediction = predict(target_user, target_item, k)
    prediction_list.append(prediction)

    print(test[i][0],test[i][1],test[i][2],prediction)

31.0 6104.0 0.11210762331838565 0.9436300550592766
119.0 38669.0 0.06756756756756757 0.2104172835188264
164.0 4597.0 0.43103448275862066 0.08620689655172414
88.0 712.0 1.2671232876712328 0.051339077283135325
121.0 14964.0 1.25 -1
116.0 7128.0 0.16393442622950818 0.14239315629858432
63.0 4096.0 0.043859649122807015 0.10386665596148736
88.0 16055.0 0.136986301369863 0.07065407807111271
185.0 2731.0 0.2631578947368421 0.14543809737536115
63.0 19500.0 0.08771929824561403 -1
80.0 7036.0 0.2702702702702703 0.13513513513513514
124.0 4.0 0.11235955056179775 0.11235955056179775
130.0 21567.0 0.24390243902439024 -1
164.0 9005.0 0.43103448275862066 0.747903847502322
134.0 300.0 0.08287292817679558 0.04054105438640667
194.0 12843.0 0.7142857142857143 0.23809523809523808
170.0 52620.0 0.5747126436781609 0.11494252873563218
134.0 44112.0 0.027624309392265192 0.12763062346376636
130.0 17281.0 0.12195121951219512 -1
83.0 43558.0 0.09803921568627451 0.7843137254901961
74.0 11972.0 0.19230769230769232 0

123.0 54249.0 0.29411764705882354 0.2605065814257728
68.0 27030.0 0.06172839506172839 0.0735343686425987
171.0 5275.0 0.272108843537415 0.06142513259789678
31.0 3468.0 0.02242152466367713 0.01680834629677771
22.0 1690.0 0.5263157894736842 0.3505089569442786
12.0 20798.0 0.15625 0.15625
182.0 422.0 0.17857142857142858 0.10165879020646341
194.0 21933.0 0.23809523809523808 0.35494129457475754
178.0 29960.0 0.027932960893854747 0.09361759790345241
99.0 29696.0 0.18518518518518517 0.23116081523883078
50.0 3073.0 3.4693877551020407 0.5204299582666125
27.0 7209.0 0.08771929824561403 0.12984801470002524
116.0 22886.0 0.32786885245901637 0.08297654352254244
10.0 5241.0 0.7142857142857143 0.7142857142857143
59.0 9986.0 0.07692307692307693 0.1413724284995697
23.0 34074.0 0.07462686567164178 0.2245238655658389
147.0 36163.0 0.08196721311475409 0.12745884480495792
145.0 19424.0 0.012953367875647668 0.012953367875647666
77.0 24677.0 0.35714285714285715 0.16240914916747629
123.0 9387.0 0.294117647058

138.0 39639.0 0.6862745098039216 0.09803921568627451
75.0 52634.0 0.6557377049180327 0.31841223283113734
214.0 10385.0 0.15384615384615385 0.07692307692307693
88.0 27673.0 0.03424657534246575 0.09831303958601934
125.0 28769.0 0.08771929824561403 0.09618890157761971
158.0 19404.0 0.019801980198019802 0.0437745226061047
148.0 46911.0 0.054945054945054944 0.11042505718038546
47.0 26516.0 0.53125 0.16672216455597214
188.0 2641.0 0.5319148936170213 0.36190343137794273
13.0 25624.0 0.14285714285714285 0.20960014306674896
174.0 36842.0 0.09615384615384616 0.3299125894421435
147.0 3099.0 0.08196721311475409 0.14468133798489474
147.0 20215.0 0.4918032786885246 0.11827678982511701
123.0 41805.0 0.39215686274509803 0.0980392156862745
52.0 3870.0 2.3353293413173652 0.06859805288775346
42.0 26926.0 2.541436464088398 -1
73.0 34992.0 0.24822695035460993 0.1316503222001448
80.0 31336.0 0.13513513513513514 -1
143.0 32017.0 0.136986301369863 -1
45.0 2980.0 0.13986013986013987 0.1817111205882295
171.0 98

61.0 242.0 0.6060606060606061 0.3461859290354915
82.0 34502.0 0.22727272727272727 0.14555205855757877
167.0 888.0 0.14354066985645933 0.0634494341562792
33.0 8360.0 0.28350515463917525 0.05119551223681132
74.0 32073.0 0.6730769230769231 0.15706373048562844
148.0 39520.0 0.054945054945054944 0.13382212049043757
15.0 3606.0 0.16129032258064516 -1
56.0 15763.0 0.06944444444444445 0.06905886210749151
172.0 26861.0 0.11904761904761904 0.14668187794869686
41.0 131.0 0.17857142857142858 0.03571428571428572
160.0 6334.0 0.07692307692307693 0.6923076923076923
6.0 8657.0 0.06802721088435375 0.06913012676152033
155.0 1800.0 0.9722222222222222 0.27781706563735203
210.0 1182.0 0.14084507042253522 0.24852333308955485
147.0 49961.0 0.08196721311475409 0.09594029321433212
45.0 27927.0 0.03496503496503497 -1
32.0 26340.0 0.43478260869565216 -1
4.0 14751.0 0.07462686567164178 0.1315409522906487
134.0 24274.0 0.027624309392265192 0.06975293881344433
52.0 10456.0 0.1497005988023952 -1
109.0 22523.0 0.2083

74.0 36160.0 0.09615384615384616 0.12211873666134845
182.0 32222.0 0.07142857142857142 0.04777660337718312
188.0 232.0 0.3191489361702128 0.18361264321146734
50.0 42162.0 0.5102040816326531 0.24246992649502544
100.0 25134.0 0.10204081632653061 0.10204081632653061
33.0 27427.0 0.02577319587628866 0.09332931494954418
188.0 2625.0 0.3191489361702128 0.18356071801165752
191.0 2704.0 0.1388888888888889 0.46849165856323155
127.0 12540.0 0.35714285714285715 0.17857142857142858
54.0 33616.0 0.14705882352941177 0.29411764705882354
182.0 47540.0 0.03571428571428571 0.09523418563293007
6.0 18640.0 0.034013605442176874 0.059483504312321175
72.0 10743.0 0.07042253521126761 0.12539658421234193
3.0 5636.0 0.2892561983471074 0.09322620420306128
172.0 4328.0 0.7142857142857143 0.11935373524399516
82.0 742.0 0.11363636363636363 0.22403103009949016
7.0 17806.0 0.1724137931034483 0.257339539648176
91.0 28628.0 0.22727272727272727 0.22727272727272724
100.0 19934.0 0.10204081632653061 0.13018871156916545
59

...

189.0 17776.0 1.2820512820512822 0.20252699256647544
198.0 12916.0 0.7692307692307693 0.38461538461538464
202.0 27451.0 0.4497354497354497 0.05291005291005291
198.0 47665.0 0.38461538461538464 -1
61.0 40739.0 0.07575757575757576 0.22115136483537234
105.0 50924.0 0.10526315789473684 0.07520758018764441
66.0 3209.0 0.6716417910447762 0.22638915220264275
179.0 3628.0 0.29069767441860467 0.15509044739447206
38.0 3555.0 0.0423728813559322 0.16949152542372883
194.0 34873.0 0.47619047619047616 -1
167.0 21338.0 0.023923444976076555 0.09529389760350714
11.0 27208.0 0.20408163265306123 -1
148.0 51607.0 0.10989010989010989 0.13894901648610866
190.0 11166.0 0.3125 0.46874999999999994
61.0 4865.0 0.22727272727272727 0.1800797087724399
49.0 11101.0 0.5 1.1017895093315555
72.0 8286.0 0.14084507042253522 0.1055635105272363
47.0 32362.0 0.03125 0.13914908091310446
144.0 38509.0 0.10204081632653061 0.336617464096083
101.0 21574.0 0.09259259259259259 0.12021543687844057
148.0 1203.0 0.38461538461538464 0

16.0 986.0 0.11363636363636363 0.05224622412138357
200.0 920.0 0.6818181818181818 -1
57.0 12510.0 0.06048387096774194 0.040322580645161296
151.0 7295.0 0.4166666666666667 0.6969774076060405
147.0 42177.0 0.08196721311475409 0.21967705012729646
192.0 43944.0 0.5555555555555556 -1
148.0 21872.0 0.10989010989010989 0.09799063158020005
50.0 30407.0 0.30612244897959184 0.21892728339104195
72.0 46656.0 0.07042253521126761 0.07042253521126761
148.0 12447.0 0.054945054945054944 0.09242703950009187
66.0 4312.0 0.07462686567164178 0.2694689106375775
155.0 23845.0 0.2777777777777778 0.2777777777777778
148.0 30920.0 0.10989010989010989 0.09949428539893501
178.0 12381.0 0.25139664804469275 0.07487668890776224
72.0 33782.0 0.07042253521126761 0.1397285145122856
33.0 4141.0 0.10309278350515463 0.09798421212361537
88.0 22108.0 0.2054794520547945 0.10603953172955359
38.0 12436.0 0.2542372881355932 0.2026092299631059
196.0 12699.0 0.375 0.15734305748471267
138.0 18662.0 0.09803921568627451 0.12846647156

68.0 50927.0 0.06172839506172839 0.09247437325748024
50.0 23579.0 0.20408163265306123 0.3391313113559838
4.0 15008.0 0.07462686567164178 0.12047379340989942
71.0 49987.0 0.6481481481481481 0.17656147607388342
218.0 19055.0 0.008880994671403197 -1
88.0 2734.0 0.3082191780821918 0.17296553399331258
205.0 28793.0 0.5 -1
124.0 9904.0 1.348314606741573 0.2736042698324143
164.0 42653.0 0.04310344827586207 0.10768954104691966
148.0 35099.0 0.16483516483516483 0.13121759421802978
147.0 43304.0 0.08196721311475409 0.09349734715522379
148.0 45756.0 0.10989010989010989 0.1173439982391783
24.0 37219.0 0.35714285714285715 -1
113.0 11382.0 0.08928571428571429 0.17857142857142858
151.0 7311.0 0.4166666666666667 0.10416666666666669
188.0 28858.0 0.7446808510638298 0.2568598284293308
179.0 34265.0 0.05813953488372093 0.10471073882697832
177.0 27719.0 0.08278145695364239 0.0796014607632351
55.0 48146.0 0.8620689655172413 0.39306096471553265
81.0 8876.0 0.20833333333333334 0.05208333333333333
41.0 12855.

164.0 34098.0 0.25862068965517243 0.24130300245513095
41.0 35541.0 0.07142857142857142 0.03571428571428571
102.0 3335.0 0.2 0.6
68.0 35830.0 1.5432098765432098 0.06172839506172839
66.0 14721.0 0.14925373134328357 0.20694354301975543
86.0 14449.0 0.15625 0.2204652235741151
179.0 35266.0 0.05813953488372093 0.1400708883160846
40.0 37548.0 0.38461538461538464 0.19230769230769232
37.0 21450.0 0.5769230769230769 -1
209.0 12140.0 0.38 0.525238535874212
188.0 36713.0 0.5319148936170213 0.2109927180300013
72.0 4933.0 0.07042253521126761 0.07647551617745142
31.0 35497.0 0.033632286995515695 0.04304187316115836
161.0 44607.0 0.21739130434782608 -1
100.0 34281.0 0.10204081632653061 0.13433182596514234
72.0 3319.0 0.352112676056338 0.11727441071920777
170.0 43467.0 0.05747126436781609 0.08622946853374437
134.0 35633.0 0.19337016574585636 0.2180449826227241
95.0 19220.0 0.5882352941176471 0.14705882352941177
143.0 7851.0 0.3424657534246575 0.0684931506849315
81.0 43550.0 0.78125 1.008707215150272
4

196.0 16140.0 0.375 0.22485881140298997
3.0 22.0 0.04132231404958678 0.0821195263482949
196.0 34753.0 0.125 0.125
167.0 3748.0 0.023923444976076555 0.05608507871055795
100.0 37123.0 0.10204081632653061 0.21002744311456162
33.0 23248.0 0.02577319587628866 0.1323202571416534
100.0 20150.0 0.10204081632653061 0.18929933476515035
196.0 51839.0 0.25 0.21716339794257908
182.0 3353.0 0.03571428571428571 0.0688637093080768
188.0 1969.0 0.425531914893617 0.19204213138760579
115.0 23939.0 0.02631578947368421 0.07244979524083375
211.0 32121.0 0.20833333333333334 0.22656743104917448
4.0 2325.0 0.14925373134328357 0.2546811442408682
213.0 52581.0 0.2857142857142857 0.1521729711164001
196.0 18147.0 0.125 0.26859253439709735
7.0 20577.0 0.1724137931034483 0.21284496386936147
4.0 7498.0 0.07462686567164178 0.08965284332154427
71.0 34304.0 0.09259259259259259 0.09259259259259259
115.0 26541.0 1.0263157894736843 0.4595968070852517
166.0 28943.0 0.1282051282051282 0.1282051282051282
126.0 15687.0 0.07812